## Imports

In [1]:
from geopy.geocoders import GoogleV3
from bs4 import BeautifulSoup
import requests
import time
import pandas as pd
import numpy as np
import urllib3
import re
from textwrap import shorten
import os

In [2]:
df = pd.read_csv('Chicago_dev_popup_pin - Sheet1.csv')

## Image URL collector

In [111]:
# Drop last n rows of a df. In case there are sums that need clipped
# df.drop(df.tail(n).index, inplace=True)

In [3]:
df['image_links'] = ''
links = []
for url in df['story_link']:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')
        images = soup.find_all('img')
        counter = 0
        for image in images:
            counter += 1
            if 'jpg' in image['src']:
                links.append(image['src'])
                if counter == len(images):
                    links.append('no image found')
                break
    except:
        links.append('no_image')

df['image_links'] = links

## Article link formatter

In [4]:
len(links)

24

In [5]:
df['developers']

0                                               Bally's
1                                                  Onni
2                                               Shapack
3                                       Shapack and CRG
4                                  Humboldt Park Health
5                                            DL3 Realty
6                                             Fern Hill
7                                     Draper & Kramer's
8     Maverk Development, GW Properties, Luxury Livi...
9                                    Community Builders
10                          Gloub & Co, GSP Development
11                                        RDM Companies
12                                      548 Development
13                           G Carter Group, P3 Markets
14                                                LUCHA
15                                              Shapack
16         Chicago Lighthouse and Brinshore Development
17    Michaels Organization and Northbrook-based

In [6]:
df["short_description"] = df["developers"].apply(lambda s: shorten(s, width=20, placeholder=""))
df['remaining_desc'] = df.apply(lambda row : row['developers'].replace(str(row['short_description']), ''), axis=1)
df['remaining_desc']

df["description_link"] = '<a href="' + df["story_link"] + '">' + df["short_description"] + "</a>" + df["remaining_desc"]
df['description_link']

0     <a href="https://therealdeal.com/chicago/2022/...
1     <a href="https://therealdeal.com/chicago/2022/...
2     <a href="https://therealdeal.com/chicago/2022/...
3     <a href="https://therealdeal.com/chicago/2022/...
4     <a href="https://therealdeal.com/chicago/2022/...
5     <a href="https://therealdeal.com/chicago/2022/...
6     <a href="https://therealdeal.com/chicago/2022/...
7     <a href="https://therealdeal.com/chicago/2022/...
8     <a href="https://therealdeal.com/chicago/2022/...
9     <a href="https://therealdeal.com/chicago/2022/...
10    <a href="https://therealdeal.com/chicago/2022/...
11    <a href="https://therealdeal.com/chicago/2022/...
12    <a href="https://therealdeal.com/chicago/2022/...
13    <a href="https://therealdeal.com/chicago/2022/...
14    <a href="https://therealdeal.com/chicago/2022/...
15    <a href="https://therealdeal.com/chicago/2022/...
16    <a href="https://therealdeal.com/chicago/2022/...
17    <a href="https://therealdeal.com/chicago/2

In [7]:
df.description_link[0]

'<a href="https://therealdeal.com/chicago/2022/05/25/ballys-wins-city-council-approval-for-chicagos-first-casino/">Bally\'s</a>'

## Google Maps API Geolocater Setup

In [8]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [9]:
df['geo_address'] = df['full_address'] + ' Chicago, IL'
df['loc'] = df['geo_address'].apply(geolocator.geocode, timeout=10)
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

## Correction Section

In [10]:
df.columns

Index(['full_address', 'developers', 'land_sale_price', 'project_price',
       'tax_increment_financing_and_low_income_tax_credits',
       'total_num_of_resi_units', 'affordable_housing', 'acerage', 'status',
       'description', 'story_link', 'image_links', 'short_description',
       'remaining_desc', 'description_link', 'geo_address', 'loc', 'point',
       'lat', 'lon', 'altitude'],
      dtype='object')

In [11]:
df

,full_address,developers,land_sale_price,project_price,tax_increment_financing_and_low_income_tax_credits,total_num_of_resi_units,affordable_housing,acerage,status,description,...,image_links,short_description,remaining_desc,description_link,geo_address,loc,point,lat,lon,altitude
0,777 West Chicago Avenue,Bally's,NaN,1.740000e+09,NaN,NaN,NaN,30.0,planned,"casino and hotel, a 3,000-seat theater, an out...",...,https://s11.therealdeal.com/trd/ci/up/2022/05/...,Bally's,,"<a href=""https://therealdeal.com/chicago/2022/...","777 West Chicago Avenue Chicago, IL","(777 W Chicago Ave, Chicago, IL 60654, USA, (4...","(41.8954383, -87.6456267, 0.0)",41.895438,-87.645627,0.0
1,700 West Chicago Avenue,Onni,45000000.0,NaN,NaN,NaN,NaN,7.0,NaN,unknown but likely residential,...,https://s13.therealdeal.com/trd/ci/up/2022/06/...,Onni,,"<a href=""https://therealdeal.com/chicago/2022/...","700 West Chicago Avenue Chicago, IL","(700 W Chicago Ave, Chicago, IL 60654, USA, (4...","(41.8970891, -87.6461563, 0.0)",41.897089,-87.646156,0.0
2,509 North Union Avenue,Shapack,25000000.0,NaN,NaN,NaN,NaN,NaN,NaN,unknown but likely a hotel,...,https://s13.therealdeal.com/trd/ci/up/2022/05/...,Shapack,,"<a href=""https://therealdeal.com/chicago/2022/...","509 North Union Avenue Chicago, IL","(509 N Union Ave, Chicago, IL 60654, USA, (41....","(41.8916396, -87.6455494, 0.0)",41.891640,-87.645549,0.0
3,170 North Green Street,Shapack and CRG,NaN,2.090000e+08,NaN,275.0,55.0,NaN,proposed,"mixed-use: residential, retail, office",...,https://s12.therealdeal.com/trd/ci/up/2022/04/...,Shapack and CRG,,"<a href=""https://therealdeal.com/chicago/2022/...","170 North Green Street Chicago, IL","(170 N Green St, Chicago, IL 60607, USA, (41.8...","(41.8850831, -87.6490542, 0.0)",41.885083,-87.649054,0.0
4,1044 North Francisco Avenue,Humboldt Park Health,NaN,NaN,NaN,300.0,300.0,NaN,proposed,affordable housing,...,https://s12.therealdeal.com/trd/ci/up/2022/06/...,Humboldt Park Health,,"<a href=""https://therealdeal.com/chicago/2022/...","1044 North Francisco Avenue Chicago, IL","(1044 N Francisco Ave, Chicago, IL 60622, USA,...","(41.9006786, -87.6996714, 0.0)",41.900679,-87.699671,0.0
5,914 West 63rd Street,DL3 Realty,NaN,2.840000e+07,NaN,108.0,10.0,NaN,proposed,mixed-income housing,...,https://s13.therealdeal.com/trd/ci/up/2022/05/...,DL3 Realty,,"<a href=""https://therealdeal.com/chicago/2022/...","914 West 63rd Street Chicago, IL","(914 W 63rd St, Chicago, IL 60621, USA, (41.77...","(41.779898, -87.648078, 0.0)",41.779898,-87.648078,0.0
6,12 West Maple Street,Fern Hill,NaN,NaN,NaN,54.0,NaN,NaN,planned,condos,...,https://s11.therealdeal.com/trd/ci/up/2022/05/...,Fern Hill,,"<a href=""https://therealdeal.com/chicago/2022/...","12 West Maple Street Chicago, IL","(12 W Maple St, Chicago, IL 60610, USA, (41.90...","(41.9021483, -87.6288955, 0.0)",41.902148,-87.628895,0.0
7,330 West Chestnut Street,Draper & Kramer's,NaN,NaN,NaN,129.0,NaN,NaN,planned,apartments,...,https://s13.therealdeal.com/trd/ci/up/2022/04/...,Draper & Kramer's,,"<a href=""https://therealdeal.com/chicago/2022/...","330 West Chestnut Street Chicago, IL","(330 W Chestnut St, Chicago, IL 60610, USA, (4...","(41.8983875, -87.6370791, 0.0)",41.898387,-87.637079,0.0
8,535 North Clair Street,"Maverk Development, GW Properties, Luxury Livi...",NaN,NaN,NaN,248.0,NaN,NaN,planned,apartments,...,https://s11.therealdeal.com/trd/ci/up/2022/04/...,"Maverk Development,","GW Properties, Luxury Living Chicago","<a href=""https://therealdeal.com/chicago/2022/...","535 North Clair Street Chicago, IL","(535 N St Clair St, Chicago, IL 60611, USA, (4...","(41.8919731, -87.6221473, 0.0)",41.891973,-87.622147,0.0
9,4715 North Western Avenue,Community Builders,NaN,NaN,NaN,51.0,51.0,NaN,proposed,affordable housing/possibly stalled,...,https://s11.therealdeal.com/trd/ci/up/2022/04/...,Community Builders,,"<a href=""https://therealdeal.com/chicago/2022/...","4715 North Western Avenue Chicago, IL","(4715 N Western Ave, Chicago, IL 60625,

## Correction section

In [121]:
# df.at[0,'Net decline in footprint, in sf']=('Almost 200K sf')
# df.at[1,'Address(es) of space company vacated or plans to vacate, in sf']=('690 East Middlefield Road, Mountain View. Synopsys is subletting the entire building to Waymo.')

## HTML popup formatter

In [12]:
df.columns

Index(['full_address', 'developers', 'land_sale_price', 'project_price',
       'tax_increment_financing_and_low_income_tax_credits',
       'total_num_of_resi_units', 'affordable_housing', 'acerage', 'status',
       'description', 'story_link', 'image_links', 'short_description',
       'remaining_desc', 'description_link', 'geo_address', 'loc', 'point',
       'lat', 'lon', 'altitude'],
      dtype='object')

In [17]:
def popup_html(row):
    i = row
    address = df['full_address'].iloc[i]
    developer = df['description_link'].iloc[i]
    description = df['description'].iloc[i]
    image = df['image_links'].iloc[i]
    
    html = '''<!DOCTYPE html>
    <html>
    <img src={} width="256" height="156">'''.format(image) + '''<br>______________________________________<br>
    Developer: <strong>{}'''.format(developer) + '''</strong><br>
    Address: <em>{}'''.format(address) + '''</em><br>
    Description: <strong><em>{}'''.format(description) + '''</strong></em><br>
    </html>
    '''
    return html


### HTML reservoir

In [124]:
#     Developer(s): <strong><em>{}'''.format(developer) + '''</strong></em><br><br>

## Map Maker

In [125]:
df.description_link[0]

'<a href="https://therealdeal.com/sanfrancisco/2022/04/27/nektar-therapeutics-looks-to-sublease-200k-sf-in-mission-bay/">Nektar Therapeutics</a> '

In [25]:
import folium
import branca

f = folium.Figure(width=750, height=750)
m = folium.Map(location=df[["lat", "lon"]].mean().to_list(),zoom_start=10)

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Proposed and planned Chicago residential development since Jan. 1')

for i in range(0,len(df)):
    html = popup_html(i)
    iframe = branca.element.IFrame(html=html)
    popup = folium.Popup(folium.Html(html, script=True))
    folium.Marker([df['lat'].iloc[i],df['lon'].iloc[i]],
                 popup=popup).add_to(m)

m.get_root().html.add_child(folium.Element(title_html))
m.fit_bounds(bounds=df[['lat','lon']].mean().to_list(),max_zoom=15)
# m.add_to(f)
m

In [127]:
m.save('index.html')

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [128]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [129]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/bay_area_company_downsize
